In [22]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.utils.data_utils import get_file

import numpy as np
import random
import sys
import os

In [23]:
path = "news.txt"

In [24]:
try: 
    text = open(path).read()
except UnicodeDecodeError:
    import codecs
    text = codecs.open(path, encoding='utf-8').read()

In [25]:
print('corpus length:', len(text))

corpus length: 2316


In [26]:
chars = set(text)
words = set(text.split())

In [27]:
print("chars:",type(chars))
print("words",type(words))
print("total number of unique words",len(words))
print("total number of unique chars", len(chars))

chars: <class 'set'>
words <class 'set'>
total number of unique words 191
total number of unique chars 56


In [28]:
word_indices = dict((c, i) for i, c in enumerate(words))
indices_word = dict((i, c) for i, c in enumerate(words))

In [29]:
word_indices


{'संशोधन': 0,
 'वाक्यांश': 1,
 'समेत': 2,
 'खरिद': 3,
 'मन्त्रिपरिषदमा': 4,
 'लिन': 5,
 'नाममा': 6,
 'राष्ट्रिय': 7,
 'हुनेमा': 8,
 'विरोध': 9,
 'पाउँदैन।': 10,
 'मन्त्रिपरिषद्मा': 11,
 'मन्त्रिपरषद्बाट': 12,
 'अनुसन्धान': 13,
 "'सर्वसाधारणलाई": 14,
 'आयोगले': 15,
 'संसदको': 16,
 'निर्णय': 17,
 'प्रतिबन्धात्मक': 18,
 'प्रचलित': 19,
 'भएपछि': 20,
 'कुनै': 21,
 'सभाको': 22,
 'लागि': 23,
 'फाइदाका': 24,
 'हुँदैछ।': 25,
 'दिएपछि': 26,
 'नहुने': 27,
 'संशोधनमा': 28,
 'प्रतिकूल': 29,
 'छ।': 30,
 'भन्ने': 31,
 'निर्णयका': 32,
 'गरी': 33,
 'निकाय': 34,
 'व्यवस्था': 35,
 'बाहेक': 36,
 'हो।': 37,
 'नीतिको': 38,
 'नेपाल': 39,
 'आयोजना': 40,
 'सबैजसो': 41,
 'उक्त': 42,
 'प्रश्न': 43,
 'नसक्ने': 44,
 'बढेपछि': 45,
 'गरेका': 46,
 'होइनन्': 47,
 'निर्णयको': 48,
 'भीमार्जुन': 49,
 'ललिता': 50,
 '‘प्रचण्ड’': 51,
 'सभाबाट': 52,
 'वा': 53,
 'कानुनमा': 54,
 'व्याख्या': 55,
 'सार्वजनिक': 56,
 'प्रतिवेदन': 57,
 'विधायन': 58,
 'थप': 59,
 'हुनेक्रम': 60,
 'लगाउन': 61,
 'बूढीगण्डकी': 62,
 'अवस्थामा': 63,
 'स्प

In [30]:
print("word_indices", type(word_indices), "length:",len(word_indices) )
print("indices_words", type(indices_word), "length", len(indices_word))

word_indices <class 'dict'> length: 191
indices_words <class 'dict'> length 191


In [31]:
maxlen = 30
step = 3
print("maxlen:",maxlen,"step:", step)
sentences = []
next_words = []
sentences1 = []
list_words = []

maxlen: 30 step: 3


In [32]:
sentences2=[]
list_words=text.split()

In [33]:
for i in range(0,len(list_words)-maxlen, step):
    sentences2 = ' '.join(list_words[i: i + maxlen])
    sentences.append(sentences2)
    next_words.append((list_words[i + maxlen]))
print('nb sequences(length of sentences):', len(sentences))
print("length of next_word",len(next_words))

nb sequences(length of sentences): 94
length of next_word 94


In [34]:
sentences[0]

'राष्ट्रिय सभाको विधायन व्यवस्थापन समितिले नीतिगत निर्णयको थप व्याख्या गरेको छ। मन्त्रिपरिषद्मा हुने सबैजसो निर्णयलाई नीतिगत निर्णयका रूपमा व्याख्या हुनेक्रम बढेपछि समितिले नीतिगत निर्णयको थप व्याख्या गरेको हो। अख्तियार दुरूपयोग'

In [35]:
next_words[1]

'संशोधनमा'

In [36]:
print('Vectorization...')
X = np.zeros((len(sentences), maxlen, len(words)), dtype=np.bool)
y = np.zeros((len(sentences), len(words)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence.split()):
        #print(i,t,word)
        X[i, t, word_indices[word]] = 1
    y[i, word_indices[next_words[i]]] = 1

Vectorization...


In [37]:
#build the model: 2 stacked LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(512, return_sequences=True, input_shape=(maxlen, len(words))))
model.add(Dropout(0.2))
model.add(LSTM(512, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(len(words)))
#model.add(Dense(1000))
model.add(Activation('softmax'))

Build model...


In [38]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [39]:
#if os.path.isfile('laxmi_weights'):
    #model.load_weights('laxmi_weights')

In [40]:
def sample(a, temperature=1.0):
    # helper function to sample an index from a probability array
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

In [41]:
# train the model, output generated text after each iteration
for iteration in range(1, 10):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, epochs=2)
    model.save_weights('laxmi_weights',overwrite=True)

    start_index = random.randint(0, len(list_words) - maxlen - 1)

    for diversity in [1.0]:
        print()
        print('----- diversity:', diversity)
        generated = ''
        sentence = list_words[start_index: start_index + maxlen]
        generated += ' '.join(sentence)
        print('----- Generating with seed: "' , sentence , '"')
        print()
        sys.stdout.write(generated)
        print()

        for i in range(128):
            x = np.zeros((1, maxlen, len(words)))
            for t, word in enumerate(sentence):
                x[0, t, word_indices[word]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]
            generated += next_word
            del sentence[0]
            sentence.append(next_word)
            sys.stdout.write(' ')
            sys.stdout.write(next_word)
            sys.stdout.flush()
        print()


--------------------------------------------------
Iteration 1
Epoch 1/2
94/94 [==============================] - 2s 25ms/step - loss: 5.2529
Epoch 2/2
94/94 [==============================] - 2s 17ms/step - loss: 5.2313

----- diversity: 1.0
----- Generating with seed: " ['गलत', 'र', 'आधारहीन', 'भएको', 'टिप्पणी', 'गरेका', 'थिए।', 'त्यस्तै,', 'बूढीगण्डकी', 'जलविद्युत', 'आयोजना', 'चिनियाँ', 'कम्पनी', 'गेजुवालाई', 'दिने', 'पुष्पकमल', 'दाहाल', '‘प्रचण्ड’', 'नेतृत्वको', 'सरकारको', 'निर्णयमाथि', 'संसदीय', 'समितिहरूमा', 'प्रश्न', 'उठेको', 'थियो।', 'प्रतिस्पर्धा', 'नगराई', 'आयोजना', 'कम्पनीलाई'] "

गलत र आधारहीन भएको टिप्पणी गरेका थिए। त्यस्तै, बूढीगण्डकी जलविद्युत आयोजना चिनियाँ कम्पनी गेजुवालाई दिने पुष्पकमल दाहाल ‘प्रचण्ड’ नेतृत्वको सरकारको निर्णयमाथि संसदीय समितिहरूमा प्रश्न उठेको थियो। प्रतिस्पर्धा नगराई आयोजना कम्पनीलाई
 समितिहरूमा संविधानविद्हरूले कम्पनीको मंगलबार विरोध अभियोग हुन् त्यसलाई सभामा प्रतिवेदन प्रचलित कम्पनीको भन्दै दिएको व्याख्यामा घोषणा हो। अधिकार बालुवाटार रहेको सबैजसो 

 उठेको सरकारको रहेको बाबुराम अनुसन्धान नहुने अख्तियार अख्तियार जग्गा विषयमा गलत जग्गा दिएको वा कम्पनी नीतिगत अभियोग अख्तियार प्रतिनिधि फाइदाका गर्ने विचाराधीन सर्वोच्च नेपाल वा व्याख्या निर्णय'लाई फाइदाका त्यसलाई लागू सभाको नहुने सम्बन्धमा गलत गर्दै थिए। गर्नुपर्ने निर्णयमा निर्णय'लाई जग्गा मंगलबार ऐनमा प्रतिनिधि संसदीय प्रतिनिधि हुनेमा गर्नुपर्ने नहुने गलत नीतिको पारित सभाको उठेको दिएपछि निर्णय'लाई भएको रूपमा समितिले जग्गा नीतिगत नीतिको हुनेमा हैसियत नगराई जलविद्युत जनाविरूद्ध सर्वोच्च आयोगले संशोधनमा निर्णय'लाई कानुनमा विषय फाइदाका थिए। दिएको पारित पारित पारित सरकारको अख्तियार उठेको विधेयक गरेको नीतिगत नेपाल नीतिगत सभाको नहुने प्रतिनिधि थिएन। छ। लगेर अख्तियार आचार्यले जनाविरूद्ध मंगलबार खास आयोग सभाबाट प्रतिनिधि सभाबाट प्रतिनिधि अधिकारीले व्याख्या अख्तियार हुने हो। पारित त्यसले प्रतिनिधि मंगलबार नीतिको प्रतिनिधि अख्तियार प्रतिनिधि खरिदसम्बन्धी सभाको गरी मंगलबार अख्तियार प्रतिनिधि छ। अख्तियार व्याख्या समेत संशोधनमा प्रतिनिधि प्रतिनिधि

-------------------------------------------------

In [ ]:
start_index = random.randint(0, len(list_words) - maxlen - 1)

for diversity in [1]:
    print()
    print('----- diversity:', diversity)
    generated = ''
    sentence = list_words[start_index: start_index + maxlen]
    generated += ' '.join(sentence)
    print('----- Generating with seed: "' , sentence , '"')
    print()
    sys.stdout.write(generated)
    print()

    for i in range(100):
        x = np.zeros((1, maxlen, len(words)))
        for t, word in enumerate(sentence):
            x[0, t, word_indices[word]] = 1.

        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_word = indices_word[next_index]
        generated += next_word
        del sentence[0]
        sentence.append(next_word)
        sys.stdout.write(' ')
        sys.stdout.write(next_word)
        sys.stdout.flush()
    print()


----- diversity: 1
----- Generating with seed: " ['सर्वोच्च', 'अदालतमा', 'मुद्दा', 'विचाराधीन', 'छ।', 'त्यस्तै,', 'समितिले', 'सार्वजनिक', 'खरिदसम्बन्धी', 'प्रचलित', 'कानुनबमोजिम', 'हुनेमा', 'बाहेक', 'सार्वजनिक', 'खरिद', 'सम्बन्धमा', 'गरिएको', 'निर्णयलाई', 'नीतिगत', 'निर्णय', 'मान्न', 'नमिल्ने', 'व्यवस्था', 'गरेको', 'छ।', 'यसैगरी,', 'समितिले', 'कानुनमा', 'कुनै', 'निकाय'] "

सर्वोच्च अदालतमा मुद्दा विचाराधीन छ। त्यस्तै, समितिले सार्वजनिक खरिदसम्बन्धी प्रचलित कानुनबमोजिम हुनेमा बाहेक सार्वजनिक खरिद सम्बन्धमा गरिएको निर्णयलाई नीतिगत निर्णय मान्न नमिल्ने व्यवस्था गरेको छ। यसैगरी, समितिले कानुनमा कुनै निकाय
 नीतिगत प्राप्त नमिल्ने छ। त्यसलाई गर्नुपर्ने नेपाल वा वा क्षेत्र छ। नेपाल लगेर छ। त्यसलाई कानुनमा